# Worksheet Solutions

## 2.2c

Complete the following code block to repeat the same procedure for MPI-ESM-LR:

In [ ]:
# Print the current MPI-ESM-LR cube units
print(f'The current unit for data is: {mpiesm.units}')

# convert units to kg m-2 day-1
mpiesm.convert_units('kg m-2 day-1')


# Rename the units to mm day-1. Recall that 1 kg m-2 is equivalent to 1 mm of rain
mpiesm.units = 'mm day-1'

# Save the new cube as a new netCDF file using the `outfile` filename we've provided below!
outfile = os.path.join(OUTDIR, 'mpi-esm-lr.mon.1986_2005.GERICS-REMO2015.pr.mmday-1.nc')

iris.save(mpiesm, outfile)

## 2.3e

<b>Answer:</b> Write the line of code required to calculate CHIRPS's (a) standard deviation and (b) annual maximum rainfall in the code block below. <br>
<b>Hint</b>: How could you adapt <code>chirps_ond.aggregated_by(['seasons'], iris.analysis.MEAN)</code> from above? You can refer to the [Iris documentation](https://scitools.org.uk/iris/docs/v2.4.0/iris/iris/analysis.html) if needed.

In [ ]:
# From chirps, calculate: 
# (a) chirps_std 
chirps_std = chirps_ond.aggregated_by(['seasons'], iris.analysis.STD_DEV) 

# (b) chirps_max
chirps_std = chirps.collapsed('time', iris.analysis.MAX) 


# 3.1a

In [ ]:
# Enter the ordered latitude and longitude coordinates for Kuala Lumpur here:
# lon=( , ) # longitude (East - West extent)
# lat=( , ) # latuitude (South - North extent)
lon = (99.5, 100.5)
lat = (13.5, 14.5)

# 3.2e part 1

e) Find the multi-annual monthly means (1986-2005) over the BK area for CHIRPS observations by following the same methodology as above in step d). As before we've given you the input and output files names.


In [ ]:
# Load the BK extracted data created in previous step
infile = os.path.join(CHIRPSDIR, 'chirps.mon.1986_2005.BK.nc')
data = iris.load_cube(infile)

# Add monthly coord categorisation to the time dim coordinate
iris.coord_categorisation.add_month_number(data, 'time', name='month_number')

# Now calculate monthly means
monthly_mean = data.aggregated_by(['month_number'], iris.analysis.MEAN)

# and change units to mm/day
monthly_mean.convert_units('mm day-1')

# create the area averaged monthly mean of daily rainfall
# Find latitude weights
monthly_mean.coord('longitude').guess_bounds()
monthly_mean.coord('latitude').guess_bounds()
grid_areas = iris.analysis.cartography.area_weights(monthly_mean)
# Calculate area averaged monthly mean rainfall
monthly_mean = monthly_mean.collapsed(['longitude', 'latitude'], iris.analysis.MEAN, 
                                      weights=grid_areas)


# Save output
outfile = os.path.join(CLIMDIR, 'chirps.mon.mean.1986_2005.mmday-1.nc')
iris.save(monthly_mean, outfile)

# 3.2e part 2
 Plot the CHIRPS observations, compare with the HadGEM2-ES and MPI-ESM-LR driven REMO2015 runs.

In [ ]:
# Load the CHIRPS cube
inpath = os.path.join(CLIMDIR,  'chirps.mon.mean.1986_2005.mmday-1.nc')
cube = iris.load_cube(inpath) 

# Quick line plot for cube 
qplt.plot(cube.coord('month_number'), cube)
plt.title('BK area averaged ' + gcm + '\n monthly average of daily rainfall')
ax = plt.gca()
ax.xaxis.set_label_text('Month Number')
ax.set_xlim(0.5, 12.5)
ax.set_ylim(0, 16)
plt.show()

# 3.3f
Question: Now that the cubes are all on the same grid, what differences do you see? <br> Complete the code template below to help answer this question.

In [ ]:
# Directory name where data is read from
indir = os.path.join(DATADIR, 'EAS-22', 'climatology')

# load HadGEM2-ES downscaled model data
hadgem2 = iris.load_cube(f'{indir}/hadgem2-es.OND.mean.1986_2005.pr.mmday-1.rg.nc')

# load MPI-ESM-LR downscaled model data
mpiesm = iris.load_cube(f'{indir}/mpi-esm-lr.OND.mean.1986_2005.pr.mmday-1.rg.nc')

# load CHIRPS data and extract region
chirps = iris.load_cube(f'{indir}/chirps.OND.mean.1986_2005.pr.mmday-1.nc')
chirps = chirps.intersection(latitude=lat, longitude=lon)

# Do some plotting!
plt.figure(figsize=(12, 10))
plt.subplot(1,3,1)
qplt.pcolormesh(hadgem2[0], vmin=0., vmax=8.)
plt.title('HadGEM2-ES')
plt.gca().coastlines()
plt.subplot(1,3,2)
qplt.pcolormesh(mpiesm[0], vmin=0., vmax=8.)
plt.title('MPI-ESM-LR')
plt.gca().coastlines()
plt.subplot(1,3,3)
qplt.pcolormesh(chirps[0], vmin=0., vmax=8.)
plt.title('CHIRPS')
plt.gca().coastlines()

plt.show()

# 4.2c

In [ ]:
time_periods = {'historical':'1961_1990', 'future':'2021_2050'}

for jobid in JOBIDS:
    for period in time_periods.keys():
        # Load the data:
        infile = os.path.join(DATADIR, period, jobid + '.mon.' + time_periods[period] + '.tm.rr.K.nc')
        data = iris.load_cube(infile)
        # In order to calculate OND mean, add a season coordinate, seperating OND from the other months:  
        iris.coord_categorisation.add_season(data, 'time', name='seasons', seasons=('jfmamjjas','ond'))

        # Extract the data for the OND season only:
        data_ond = data.extract(iris.Constraint(seasons='ond'))
        
        # Now calculate the mean over the OND season:
        ond_mean = data_ond.aggregated_by(['seasons'], iris.analysis.MEAN)
        
        # Convert the units from Kelvin to Celsius:
        ond_mean.convert_units('celsius')
        
        outfile = os.path.join(CLIMDIR, jobid + '.OND.mean.' + time_periods[period] + '.tm.C.nc')
        # save the OND mean as a netCDF ('outfile' specifies the output file name for your OND mean cube):
        iris.save(ond_mean, outfile)
        print('Saved: {}'.format(outfile))

# 4.4h

In [ ]:
# Read in the land-sea mask. 
# The cube data array has a mask associated with it which we'll use to mask out ocean points.
land_sea_mask = iris.load_cube(DATADIR + '/landmask.nc')

for jobid in JOBIDS:
    # Read in original data for baseline and future
    baselinepath = os.path.join(HISTDIR, jobid + '.mon.1961_1990.tm.rr.C.nc')
    futurepath = os.path.join(FUTRDIR, jobid + '.mon.2021_2050.tm.rr.C.nc')
    baseline = iris.load_cube(baselinepath)
    future = iris.load_cube(futurepath)
    
    # Apply land mask
    baseline.data = ma.array(baseline.data, mask=baseline.data*land_sea_mask.data.mask[np.newaxis, :,:])
    future.data = ma.array(future.data, mask=future.data*land_sea_mask.data.mask[np.newaxis, :,:])

    # Guess bounds
    for cube in [baseline, future]:
        for coord in ['grid_longitude', 'grid_latitude']:
            cube.coord(coord).guess_bounds()
    
    # Calculate mean values over land points
    baseline_land = baseline.collapsed(['grid_longitude', 'grid_latitude'], iris.analysis.MEAN,
                                      weights = iris.analysis.cartography.area_weights(baseline))
    future_land = future.collapsed(['grid_longitude', 'grid_latitude'], iris.analysis.MEAN,
                                  weights = iris.analysis.cartography.area_weights(future))

    # Save future & baseline area averaged monthly data (time series)
    baselineout = os.path.join(CLIMDIR, jobid + '.mon.1961_1990.series.tm.C.nc')
    futureout = os.path.join(CLIMDIR, jobid + '.mon.2021_2050.series.tm.C.nc')
    iris.save(baseline_land, baselineout)
    iris.save(future_land, futureout)

    # Subtract baseline from future
    diff = future_land.copy()
    diff.data = future_land.data - baseline_land.data.mean()
    diff.rename('future anomaly relative to mean historical temperature')

    # Save the area averaged monthly differences (time series)
    outpath = os.path.join(CLIMDIR, jobid + '.mon.2021_2050.anom.series.tm.C.nc')
    iris.save(diff, outpath)
    print('Saved: {}'.format(outpath))

In [ ]:
# Read in the monthly series
cahpa = iris.load_cube(CLIMDIR + '/cahpa.mon.2021_2050.anom.series.tm.C.nc')
cahpb = iris.load_cube(CLIMDIR + '/cahpb.mon.2021_2050.anom.series.tm.C.nc')
time = cahpa.coord('time')

# Plot the two model time series' on the same figure
plt.figure(figsize=(16,5))
iplt.plot(time, cahpa, label = 'cahpa')
iplt.plot(time, cahpb, label = 'cahpb')
plt.legend()
plt.suptitle('2021-2050 Temperature anomaly (relative to 1961-1990)')
plt.ylabel('Temperture change ({})'.format(cahpa.units))
plt.xlabel('Years')
plt.show()

# 5.1b

In [ ]:
# Load APHRODITE daily precipitation data:
infile = os.path.join(APHRODIR, 'aphro.day.1961_1990.nc')
obs = iris.load_cube(infile, 'daily precipitation analysis interpolated onto 0.25deg grids [mm/day]')
# Find number of days in dataset
number_aphro_days = len(obs.coord('time').points)
# Find number of wet days
obs_wetdays = obs.collapsed('time', count_nonzero)
obs_wetdays.rename('Aphrodite number of wet days (>=1mm/day) 1961-1990')
# Save ouput
outfile = os.path.join(CLIMDIR, 'aphro.wetday.nc')
iris.save(obs_wetdays, outfile)
print('Saved: {}'.format(outfile))

# Find wet days percent
obs_pcent_wetdays = iris.analysis.maths.divide(iris.analysis.maths.multiply(obs_wetdays, 100), number_aphro_days)
obs_pcent_wetdays.rename('Aphrodite percentage of wet days (>=1mm/day) 1961-1990')
outfile = os.path.join(CLIMDIR, 'aphro.wetday.pcent.nc')
iris.save(obs_pcent_wetdays, outfile)
print('Saved: {}'.format(outfile))

# 5.1e

In [ ]:
pcent_change_subset = pcent_change.intersection(longitude=(90, 137), 
                                                    latitude=(-20, 32))

# 5.2h

In [ ]:
# Create a figure of the size 12x12 inches
plt.figure(figsize=(12, 12))

# Load the 95th percentage differences in precipitation between the models and obs, and plot them.
for n, runid in enumerate(JOBIDS):
    infile = os.path.join(CLIMDIR, runid + '.day.pc95.bias.pr.mmday-1.nc')
    pc95_diff = iris.load_cube(infile)
    plt.subplot(2, 2, n+1)
    qplt.pcolormesh(pc95_diff.intersection(longitude=(90, 137), latitude=(-20, 32)), 
                    vmax=30, vmin=-30, cmap='BrBG')
    plt.title('{} R95p precipitation bias (model - obs) (mm day-1)'.format(runid))
    ax = plt.gca()              # gca function that returns the current axes
    ax.coastlines()             # adds coastlines defined by the axes of the plot

    # Load the 95th percentage change in precipitation between future and baseline, plot.
    infile = os.path.join(CLIMDIR, runid + '.day.pc95.diff.pr.mmday-1.nc')
    pc95_diff = iris.load_cube(infile)
    plt.subplot(2, 2, n+3)
    qplt.pcolormesh(pc95_diff, vmax=10, vmin=-10, cmap='BrBG')
    plt.title('{} R95p precipitation difference (future-historic) (mm day-1)'.format(runid))
    ax = plt.gca()      
    ax.coastlines()             

plt.tight_layout()
plt.show()